In [2]:
!pip install firebase-admin
from pandas.core.indexes.timedeltas import timedelta_range
import sys
import firebase_admin
from firebase_admin import firestore
from firebase_admin import credentials
import pandas as pd
import sqlite3
import datetime

In [3]:
import json
with open('secret.json', 'r') as f:
    data = json.load(f)

In [53]:
from sshtunnel import SSHTunnelForwarder
import pymysql

class DBHelper:

    def __init__(self):
        self.db_host = "localhost"
        self.db_user = data["db_user"]
        self.db_password = data["db_user_pass"]
        self.db_name = "asahi1958_flightdata"
        
        self.ssh_host = 'sv14816.xserver.jp'
        self.ssh_port = 10022
        self.ssh_user = 'asahi1958'
        self.ssh_pkey = 'asahi1958.key'
        self.ssh_private_key_password = data["ssh_key_pass"]
        self.ssh_mysql_host = 'localhost'
        self.ssh_mysql_port = 3306
        
    def __connect__(self):
        self.server = SSHTunnelForwarder(
            (self.ssh_host, self.ssh_port),
            ssh_username=self.ssh_user,
            ssh_pkey=self.ssh_pkey,
            ssh_private_key_password=self.ssh_private_key_password,
            remote_bind_address=(self.ssh_mysql_host,self.ssh_mysql_port)
        )
        self.server.start()
        self.con = pymysql.connect(
            host=self.db_host,
            port=self.server.local_bind_port,
            user=self.db_user,
            passwd=self.db_password,
            db=self.db_name,
            # Select結果をタブルではなく辞書で受け取る
            cursorclass=pymysql.cursors.DictCursor
        )
        self.cur = self.con.cursor()

    def __disconnect__(self):
        self.con.commit()
        self.con.close()
        self.server.stop()

    def fetch(self, sql):
        self.__connect__()
        self.cur.execute(sql)
        result = self.cur.fetchall()
        self.__disconnect__()
        return result

    def execute(self, sql, data):
        self.__connect__()
        self.cur.execute(sql, data)
        self.__disconnect__()

In [54]:
db = DBHelper()
db.fetch('CREATE TABLE flight_record(datetime DATETIME PRIMARY KEY,latitude REAL, longtitude REAL, altitude REAL, pressure REAL, temperature REAL, lat_prediction REAL, lng_prediciton REAL)')


OperationalError: (1050, "Table 'flight_record' already exists")

In [56]:
db = DBHelper()

In [66]:
def recieve_data():
    #データ受信
    dt_now = datetime.datetime.now()
    data = (dt_now, 38.2, 135.2, 3.0, 300.0, 1013.0)
    #データの保存
    sql = "INSERT INTO flight_record (datetime, latitude, longtitude, altitude, temperature, pressure) values (%s, %s, %s, %s, %s, %s)"
    db.execute(sql, data)

In [67]:
recieve_data()
db.fetch('select * from flight_record')